# Moku Tools - Notebook Widgets Demo

This notebook demonstrates all the interactive widgets available in `mokutools.moku_io.notebook` with mock connections. This allows you to explore the widgets without needing an actual Moku device connected.

In [ ]:
# Import required libraries
import sys
from unittest.mock import patch, MagicMock
from IPython.display import display

# Import notebook widgets
from mokutools.moku_io.notebook import (
    select_file_widget,
    download_files_widget,
    upload_files_widget,
    delete_files_widget,
)

# Mock IP address (not used with mocks, but required for widget signatures)
MOCK_IP = "10.128.100.198"

## Setup Mock Functions

We'll create mock functions that simulate the behavior of the core Moku I/O functions without requiring an actual device connection.

In [ ]:
# Mock file list - simulating files on a Moku device
MOCK_FILES = [
    "data_20250115_120000.li",
    "data_20250115_130000.li",
    "data_20250116_090000.li",
    "data_20250116_150000.li",
    "measurement_20250115.li",
    "test_file.li",
]

def mock_list_files(ip: str, filters=None):
    """Mock function that returns a list of files."""
    files = MOCK_FILES.copy()
    if filters:
        lower_filter = [s.lower() for s in filters]
        files = [
            fname for fname in files
            if all(sub in fname.lower() for sub in lower_filter)
        ]
    return files

def mock_download(ip, patterns=None, date=None, convert=True, 
                  archive=True, output_path=None, remove_from_server=False):
    """Mock function that simulates downloading and converting files."""
    import time
    files = mock_list_files(ip)
    
    if patterns:
        if isinstance(patterns, str):
            patterns = [patterns]
        files = [f for f in files if any(pat in f for pat in patterns)]
    
    if date:
        files = [f for f in files if date in f]
    
    if not files:
        return []
    
    # Simulate processing time
    time.sleep(0.5)
    
    processed = []
    for f in files:
        processed.append(f.replace('.li', '.csv.zip' if archive else '.csv'))
    
    return processed

def mock_upload(ip, paths):
    """Mock function that simulates uploading files."""
    import time
    if isinstance(paths, str):
        paths = [paths]
    
    results = {}
    for path in paths:
        time.sleep(0.3)  # Simulate upload time
        # Simulate success for most files, failure for "error.txt"
        results[path] = "error.txt" not in path
    
    return results

def mock_delete(ip, patterns=None, delete_all=False, confirm=False):
    """Mock function that simulates deleting files."""
    if not confirm:
        raise ValueError("Deletion requires confirmation")
    
    files = mock_list_files(ip)
    
    if delete_all:
        deleted = files.copy()
    elif patterns:
        if isinstance(patterns, str):
            patterns = [patterns]
        deleted = [f for f in files if any(pat in f for pat in patterns)]
    else:
        deleted = []
    
    return deleted

print("✅ Mock functions defined")

## 1. File Selection Widget

The `select_file_widget` provides a dropdown menu to select from available files on the device.

In [ ]:
# Patch list_files to use our mock
with patch('mokutools.moku_io.notebook.list_files', side_effect=mock_list_files):
    files = mock_list_files(MOCK_IP)
    file_dropdown = select_file_widget(files)
    
    # Display the selected value when it changes
    def on_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            print(f"Selected file: {change['new']}")
    
    file_dropdown.observe(on_change)

## 2. Download Files Widget

The `download_files_widget` allows you to download files from the device, with options to convert `.li` files to `.csv` and archive them.

In [ ]:
# Example 1: Download all files matching a pattern
with patch('mokutools.moku_io.notebook.download', side_effect=mock_download):
    download_output_1 = download_files_widget(
        ip=MOCK_IP,
        file_names="data_20250115",  # Pattern to match
        convert=True,
        archive=True,
    )
    display(download_output_1)

In [ ]:
# Example 2: Download files by date
with patch('mokutools.moku_io.notebook.download', side_effect=mock_download):
    download_output_2 = download_files_widget(
        ip=MOCK_IP,
        date="20250116",  # Date filter in YYYYMMDD format
        convert=True,
        archive=False,  # Don't archive
    )
    display(download_output_2)

## 3. Upload Files Widget

The `upload_files_widget` allows you to upload local files to the device.

In [ ]:
# Example: Upload a single file
with patch('mokutools.moku_io.notebook.upload', side_effect=mock_upload):
    upload_output_1 = upload_files_widget(
        ip=MOCK_IP,
        files="local_file.li"
    )
    display(upload_output_1)

In [ ]:
# Example: Upload multiple files
with patch('mokutools.moku_io.notebook.upload', side_effect=mock_upload):
    upload_output_2 = upload_files_widget(
        ip=MOCK_IP,
        files=["file1.li", "file2.li", "error.txt"]  # Last one will simulate failure
    )
    display(upload_output_2)

## 4. Delete Files Widget

The `delete_files_widget` provides a safe way to delete files with confirmation buttons. **Click the buttons below to see the interaction!**

In [ ]:
# Example 1: Delete files matching a pattern
with patch('mokutools.moku_io.notebook.list_files', side_effect=mock_list_files), \
     patch('mokutools.moku_io.notebook.delete', side_effect=mock_delete):
    delete_files_widget(
        ip=MOCK_IP,
        file_names="20250115"  # Delete files containing this pattern
    )

In [ ]:
# Example 2: Delete all files (use with caution!)
with patch('mokutools.moku_io.notebook.list_files', side_effect=mock_list_files), \
     patch('mokutools.moku_io.notebook.delete', side_effect=mock_delete):
    delete_files_widget(
        ip=MOCK_IP,
        delete_all=True
    )

## Summary

This notebook demonstrated all four widget types available in `mokutools.moku_io.notebook`:

1. **`select_file_widget`** - Dropdown for file selection
2. **`download_files_widget`** - Download files with conversion and archiving options
3. **`upload_files_widget`** - Upload local files to the device
4. **`delete_files_widget`** - Delete files with confirmation

All widgets were demonstrated using mock functions, so no actual device connection was required. In a real scenario, you would use these widgets with an actual Moku device IP address, and the widgets would interact with the real device.